In [9]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

from random import sample
import random

import time

In [10]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [11]:
file_name = sys.argv[1] # instancia

In [12]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [13]:
def gera_particoes(N, tam_part, tam_jan):
	
	temp = tam_part-tam_jan
	subset = []
	 
	for i in range(0, N, temp):
		if i+tam_part > N:
			subset.append([k for k in range(i, N)])
		else:
			subset.append([k for k in range(i, i+tam_part)])
	
	return subset

In [14]:
MAX_CPU_TIME = 3600.0
EPSILON = 1.e-6

def relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, parts, yp_sol, yr_sol):

	try:

		#print("nova iteracao!")

		# create a new model
		model = gp.Model("clspr_rf")

		# create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		for i in range(N):
			if i < min(parts):
				# fixadas
				yp[i].lb = yp_sol[i]
				yp[i].ub = yp_sol[i]
				yr[i].lb = yr_sol[i]
				yr[i].ub = yr_sol[i]
			elif i in parts:
				# binary
				yp[i].VType = gp.GRB.BINARY
				yr[i].VType = gp.GRB.BINARY
		
		# set objective
		model.setObjective(
			gp.quicksum(PP[i]*xp[i] + HP[i]*sp[i] + FP[i]*yp[i] + PR[i]*xr[i] + HR[i]*sr[i] + FR[i]*yr[i] for i in range(N)), 
			sense=GRB.MINIMIZE
			)

		# add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(1,N))
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(1,N))
		
		model.addConstrs(xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i], SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
		
		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)
		
		model.update()

    	# turn off display
		gp.setParam('OutputFlag', 0)

		# open log file
		_ = open('std_rf.log', 'w')
		
		# export .lp/.mps
		#model.write("clspr.lp")

		#print("Press any key to continue...")
		#input()

		# Optimize model
		model.optimize()

		#objval = 0.0
		objval = model.ObjVal

		yp_val = [yp[i].X for i in range(N)]
		yr_val = [yr[i].X for i in range(N)]

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return objval, yp_val, yr_val

In [15]:
MAX_CPU_TIME = 3600.0
EPSILON = 1.e-6

def fix_and_opt(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, parts, yp_sol ,yr_sol):

    try:

        # Create a new model
        model = gp.Model("clsrp_fo")

        # Create variables
        xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
        yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
        sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
        xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
        yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")
        sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
      
        for i in range(N):
            if i not in parts:
                yp[i].lb = yp_sol[i]
                yp[i].ub = yp_sol[i]
                yr[i].lb = yr_sol[i]
                yr[i].ub = yr_sol[i]
            #else:
            #    yp[i].start = yp_sol[i]
            #    yr[i].start = yr_sol[i]
                
        # set objective
        
        #model.setObjective(
        #    gp.quicksum(PP[i]*xp[i] + HP[i]*sp[i] + FP[i]*yp[i] + PR[i]*xr[i] + HR[i]*sr[i] + FR[i]*yr[i] for i in range(N)), 
        #    sense = GRB.MINIMIZE
        #    )

        fobj = gp.quicksum(PP[i]*xp[i] for i in range(N))
        fobj += gp.quicksum(HP[i]*sp[i] for i in range(N))
        fobj += gp.quicksum(FP[i]*yp[i] for i in range(N))
        fobj += gp.quicksum(PR[i]*xr[i] for i in range(N))
        fobj += gp.quicksum(HR[i]*sr[i] for i in range(N))
        fobj += gp.quicksum(FR[i]*yr[i] for i in range(N))

        model.setObjective(fobj, sense = GRB.MINIMIZE)
        
        # add constraints
    
        model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

        model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
        
        model.addConstr(R[0] - xr[0] - sr[0] == 0)
        
        model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
        
        model.addConstrs(xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N))
        
        model.addConstrs(xr[i] - yr[i]*min(SR[0][i], SD[i][N-1],C) <= 0 for i in range(N))
        
        model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
       
        # parameters 
        model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
        model.setParam(GRB.Param.MIPGap, EPSILON)
        model.setParam(GRB.Param.Threads, 1)
        #model.setParam(GRB.Param.Cuts, -1)
        #model.setParam(GRB.Param.Presolve, -1)

        model.update()

        # turn off display
        gp.setParam('output_flag', 0)

        # Open log file
        _ = open('std_fo.log', 'w')
		
		# export .lp/.mps
        #model.write("clspr.lp")

        #print("Press any key to continue...")
        #input()	

        # optimize model
        model.optimize()

        yp_val = [yp[i].X for i in range(N)]
        yr_val = [yr[i].X for i in range(N)]

        objval = model.ObjVal

    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    return objval, yp_val, yr_val

In [17]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_std_mip_start(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, yp_sol, yr_sol):
	try:

		# create a new model
		model = gp.Model("clsr_std_mip")

		# create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		model.update()

		# set objective
		model.setObjective(gp.quicksum(
			PP[i]*xp[i] + sp[i]*HP[i] + 
			xr[i]*PR[i] + sr[i]*HR[i] + 
			yp[i]*FP[i] + yr[i]*FR[i] for i in range(N)), sense = GRB.MINIMIZE)

		# add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
		
		model.addConstrs(xp[i] - yp[i]*min(SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
		
		for i in range(N):
			yp[i].start = yp_sol[i]
			yr[i].start = yr_sol[i]
	    
		# export .lp
		#model.write(file_name+"_model.lp")

		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# turn off display
		gp.setParam('output_flag', 0)

		# open log file
		_ = open('std_mip_start.log', 'w')

		# optimize model
		model.optimize()
		
		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		print('Obj: %g' % model.ObjVal)

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return model.ObjVal, model.ObjBound, model.MIPGap, model.Runtime, model.NodeCount, tmp

In [18]:
def main(file_name, tam_part, tam_jan):

	solution_rf = 0.0
	solution_fo = 0.0
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()
	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	start_time = time.time()

	particao = gera_particoes(N, tam_part, tam_jan)

	yp_sol = [0]*N
	yr_sol = [0]*N
	
	for subset in particao:
		solution_rf, yp_sol, yr_sol = relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, subset, yp_sol, yr_sol)

	run_time_rf = time.time() - start_time

	for subset in particao:
		solution_fo, yp_sol, yr_sol = fix_and_opt(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, subset, yp_sol, yr_sol)

	run_time_fo = time.time() - start_time

	bestsol, bestbound, gap, runtime, numnode, tmp = clsr_std_mip_start(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, yp_sol, yr_sol)

	run_time = time.time() - start_time

	arquivo = open(os.path.join(result,'clsr_std_mip_start.txt'),'a')
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(run_time,2))+';'
			   +str(round(numnode,2))+';'
			   +str(round(tmp,2))+'\n')

	arquivo.close()

In [19]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,2):
			file_name = f"c{dim}_{id}.txt"
			tam_part = 5
			tam_jan = 3
			
			main(file_name, tam_part, tam_jan)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 260 rows, 312 columns and 674 nonzeros
Model fingerprint: 0xddc53cb8
Variable types: 302 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 99 rows and 99 columns
Presolve time: 0.01s
Presolved: 161 rows, 213 columns, 477 nonzeros
Variable types: 204 continuous, 9 integer (9 binary)
Found heuristic solution: objective 6579.1075856

Root relaxation: objective 5.376769e+03, 150 iterations